El GCPD recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta informacióñ se encuentra guarada en un archivo con el siguiente formato:

(fecha, id_caso, descripción, estado_caso, categoria, latitud, longitud)

Los posibels estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolucion.

Las fechas se encuentran en el formato YYYY-MM-DD.

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuales casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato (id_caso, respeusta), siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.

b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

Resolver ambas consultas utilizando Pandas.

[Resolución de colaborador de la materia](https://gist.github.com/Roj/a19d82c1b8ebcb0198dc4773752b852a)

[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqroi7u4xe)

In [ ]:
# Set-up y vista rápida de los dos sets de datos sintéticos
import pandas as pd

CASO_ABIERTO = 1
CASO_RESUELTO = 2
CASO_CERRADO = 3

BATMAN_NO_PARTICIPA = 0
BATMAN_PARTICIPA = 1

df_gcpd = pd.read_csv('../data/2018C1_GCPD.csv')
df_gcpd['fecha'] = pd.to_datetime(df_gcpd['fecha'],format='%Y-%m-%d')
display(df_gcpd)

df_gordon = pd.read_csv('../data/2018C1_gordon.csv')
display(df_gordon)

**a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.**

- Agregamos una columna que indique el trimestre
- Nos quedamos con el trimestre anterior (3) y el anterior (2)
- Pivoteamos por trimestre y categoría
    - Contamos el % de casos resueltos en cada grupo
- Filtramos las categorías donde haya aumentado ese %

In [ ]:
df_gcpd['trimestre'] = df_gcpd.fecha.dt.month % 4
df_trimestres = df_gcpd[df_gcpd.trimestre.isin([3,2])]
df_tasa_por_trimestre = df_trimestres.pivot_table(columns='trimestre',index='categoria',aggfunc=lambda x: (x==CASO_RESUELTO).mean(),values='estado_caso').fillna(0)
df_tasa_por_trimestre[df_tasa_por_trimestre[3] > df_tasa_por_trimestre[2]].index.values.tolist()

**b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).**

- Agrupamos los dos dataframes
- Nos quedamos con los delitos contra la propiedad
- Agrupamos por categoría
    - Dentro de cada grupo calculamos el % de respuesta

In [ ]:
DELITOS_CONTRA_LA_PROPIEDAD = set(['Incendio', 'Robo', 'Hurto', 'Robo de Vehículo']) 
df = df_gcpd.merge(df_gordon,how='left',on='id_caso')
df = df[df.categoria.isin(DELITOS_CONTRA_LA_PROPIEDAD)] 
df.groupby('categoria')['respuesta'].apply(lambda x: (x==BATMAN_PARTICIPA).mean()).rename('tasa de respuesta')